In [1]:
import psycopg2
import pandas as pd

In [33]:
df_item = pd.read_csv('day1_files/items.csv')
df_item

,item_id,item_description,item_status
0,1,OCTINOXATE and TITANIUM DIOXIDE,N
1,2,medroxyprogesterone acetate,N
2,3,Progesterone,Y
3,4,penicillin G potassium,N
4,5,"Corn, Cultivated Zea mays",Y
5,6,Potassium Chloride,N
6,7,PIPERACILLIN and TAZOBACTAM,Y
7,8,clonidine hydrochloride,Y
8,9,Diflorasone Diacetate,Y
9,10,"acetaminophen, dextromethorphan HBr, doxylamin...",Y


In [80]:
df_customer = pd.read_csv('day2_files/customers.csv')
df_customer

,customer_id,customer_number,customer_name,address,postal_code,city,country,country_code,telephone
0,6,72-874-9816,"Hagenes, Dach and Waters",945 Old Gate Circle,763568,Florida,Colombia,CO,668-232-3337
1,7,82-675-2054,"Harris, Ziemann and Gorczany",65026 Ridgeway Plaza,NaN,Pinghu,China,CN,534-286-1674
2,12,78-398-2946,"Spinka, Brakus",9378 Bowman Plaza,589 00,Linköping,Sweden,SE,111-933-7553
3,18,47-147-4334,Beahan-Boehm,5442 Sachtjen Court,NaN,Wanquan,China,CN,791-319-3545
4,24,98-663-8783,Zulauf-Mayert,2184 Heath Park,NaN,Tiron,China,CN,791-680-1187


In [107]:
df_order_headers = pd.read_csv('day2_files/order_headers.csv')
df_order_headers

,order_id,order_number,order_date,customer_id,order_status,currency
0,3,632-01-9775,2/13/2019,4,CANCELLED,CNY
1,7,299-20-5035,4/13/2019,24,CLOSED,CNY
2,8,417-82-7075,1/10/2019,2,CLOSED,PAB
3,11,467-98-0779,1/3/2019,13,CANCELLED,CRC
4,25,221-10-2846,4/19/2019,25,OPEN,USD
5,26,111-22-3846,4/19/2019,2,OPEN,CNY


In [128]:
df_order_lines = pd.read_csv('day2_files/order_lines.csv')
df_order_lines

,orderline_id,order_id,item_id,ship_date,promise_date,ordered_quantity
0,1,11,9,3/31/2019,3/31/2019,44
1,10,3,6,3/26/2019,3/26/2019,178
2,16,7,9,3/29/2019,3/29/2019,957
3,21,7,3,3/8/2019,3/8/2019,813
4,29,3,9,1/13/2019,1/13/2019,184
5,53,8,9,1/9/2019,1/9/2019,263
6,55,8,10,1/29/2019,1/29/2019,617
7,68,3,6,4/10/2019,4/10/2019,247
8,74,3,5,3/4/2019,3/4/2019,357
9,79,11,4,3/1/2019,3/1/2019,663


In [34]:
# Ingest or Update: Item
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

for item in df_item.values:
    item_id = item[0]
    item_description = item[1][:50]
    item_status = item[2][:1]

    # Check if item exists using a SELECT query
    check_existing_query = f"""SELECT * FROM public.item WHERE item_id = %s;"""

    cur.execute(check_existing_query, (item_id,))
    existing_item = cur.fetchone()

    if existing_item:
        # Existing record found, update it
        update_query = f"""
        UPDATE public.item
        SET item_description = %s, item_status = %s
        WHERE item_id = %s;
        """
        cur.execute(update_query, (item_description, item_status, item_id))

    else:
        insert_query = f"""INSERT INTO public.item (item_id, item_description, item_status) VALUES (%s, %s, %s);"""
        cur.execute(insert_query, (item_id, item_description, item_status))


conn.commit()

cur.close()
conn.close()

In [82]:
# Ingest or Update: customer
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

for customer in df_customer.values:
    customer_id = customer[0]
    customer_number = customer[1][:50]
    customer_name = customer[2][:1][:50] 
    address = customer[3][:100]
    postal_code = str(customer[4])[:10] if customer[4] is not None else None
    city = customer[5][:50]
    country = customer[6][:50]
    country_code = customer[7][:10] if customer[7] is not None else None
    telephone = customer[8][:50] if customer[8] is not None else None

    # Check if customer exists using a SELECT query
    check_existing_query = f"""SELECT * FROM public.customer WHERE customer_id = %s;"""

    cur.execute(check_existing_query, (customer_id,))
    existing_customer = cur.fetchone()

    if existing_customer:
        # Existing record found, update it
        update_query = f"""
        UPDATE public.customer
        SET customer_number = %s, customer_name = %s, address = %s, postal_code = %s, 
            city = %s, country = %s, country_code = %s, telephone = %s
        WHERE customer_id = %s;
        """
        cur.execute(update_query, (customer_number, customer_name, address, postal_code, city, country, country_code, telephone, customer_id))

    else:
        insert_query = f"""INSERT INTO public.customer (customer_id, customer_number, customer_name, address, 
            postal_code, city, country, country_code, telephone) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        cur.execute(insert_query, (customer_id, customer_number, customer_name, address, postal_code, city, country, country_code, telephone))


conn.commit()

cur.close()
conn.close()

In [108]:
# Ingest or Update: order_headers
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

for order in df_order_headers.values:
    order_id = order[0]
    order_number = order[1]
    order_date = order[2]
    customer_id = order[3]
    order_status = order[4]
    currency = order[5]

    # Check if order exists using a SELECT query
    check_existing_query = f"""SELECT * FROM public.order_header WHERE order_id = %s;"""

    cur.execute(check_existing_query, (order_id,))
    existing_order = cur.fetchone()

    if existing_order:
        # Existing record found, update it
        update_query = f"""
        UPDATE public.order_header
        SET order_number = %s, order_date = %s, customer_id = %s, order_status = %s, currency = %s
        WHERE order_id = %s;
        """
        cur.execute(update_query, (order_number, order_date, customer_id, order_status, currency, order_id))

    else:
        insert_query = f"""INSERT INTO public.order_header (order_id, order_number, order_date, customer_id, order_status, currency) 
            VALUES (%s, %s, %s, %s, %s, %s);"""
        cur.execute(insert_query, (order_id, order_number, order_date, customer_id, order_status, currency))


conn.commit()

cur.close()
conn.close()

In [121]:
# Ingest or Update: order_line
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

for order in df_order_lines.values:
    orderline_id = order[0]
    order_id = order[1]
    item_id = order[2]
    ship_date = order[3]
    promise_date = order[4]
    ordered_quantity = order[5]

    # Check if order exists using a SELECT query
    check_existing_query = f"""SELECT * FROM public.order_line WHERE orderline_id = %s;"""

    cur.execute(check_existing_query, (orderline_id,))
    existing_order = cur.fetchone()

    if existing_order:
        # Existing record found, update it
        update_query = f"""
        UPDATE public.order_line
        SET order_id = %s, item_id = %s, ship_date = %s, promise_date = %s, ordered_quantity = %s
        WHERE orderline_id = %s;
        """
        cur.execute(update_query, (order_id, item_id, ship_date, promise_date, ordered_quantity, orderline_id))

    else:
        insert_query = f"""INSERT INTO public.order_line (orderline_id, order_id, item_id, ship_date, promise_date, ordered_quantity) 
            VALUES (%s, %s, %s, %s, %s, %s);"""
        cur.execute(insert_query, (orderline_id, order_id, item_id, ship_date, promise_date, ordered_quantity))


conn.commit()

cur.close()
conn.close()

In [179]:
data

[[1, 'OCTINOXATE and TITANIUM DIOXIDE', 'N'],
 [2, 'medroxyprogesterone acetate', 'N'],
 [3, 'Progesterone', 'Y'],
 [4, 'penicillin G potassium', 'N'],
 [5, 'Corn, Cultivated Zea mays', 'Y'],
 [6, 'Potassium Chloride', 'N'],
 [7, 'PIPERACILLIN and TAZOBACTAM', 'Y'],
 [8, 'clonidine hydrochloride', 'Y'],
 [9, 'Diflorasone Diacetate', 'Y'],
 [10,
  'acetaminophen, dextromethorphan HBr, doxylamine succinate, phenylephrine HCl',
  'Y']]

In [169]:
def data_table(table):
    if table == 'customer':
        return df_customer.dropna(subset=['customer_number', 'customer_name', 'address', 
    'postal_code', 'city', 'country']).values.tolist()
    else:
        return eval(f"df_{table}.values.tolist()")
    
data_table('order_lines')

[[1, 11, 9, '3/31/2019', '3/31/2019', 44],
 [10, 3, 6, '3/26/2019', '3/26/2019', 178],
 [16, 7, 9, '3/29/2019', '3/29/2019', 957],
 [21, 7, 3, '3/8/2019', '3/8/2019', 813],
 [29, 3, 9, '1/13/2019', '1/13/2019', 184],
 [53, 8, 9, '1/9/2019', '1/9/2019', 263],
 [55, 8, 10, '1/29/2019', '1/29/2019', 617],
 [68, 3, 6, '4/10/2019', '4/10/2019', 247],
 [74, 3, 5, '3/4/2019', '3/4/2019', 357],
 [79, 11, 4, '3/1/2019', '3/1/2019', 663],
 [81, 25, 3, '4/19/2019', '4/30/2019', 750],
 [82, 26, 4, '4/19/2019', '4/30/2019', 200]]

In [ ]:
INSERT INTO your_table (id, column1, column2)
VALUES (1, 'value1', 'value2')
ON CONFLICT (id) DO UPDATE
SET column1 = EXCLUDED.column1, column2 = EXCLUDED.column2;

In [213]:
len(data)

12

In [235]:
import psycopg2
import datetime

# config
table = 'customer'  #item, customer, order_headers, order_lines


# Construct the SQL INSERT statement
sql_insert_item = """INSERT INTO item (item_id, item_description, item_status) VALUES (%s, LEFT(%s,50), LEFT(%s,1)) 
    ON CONFLICT (item_id) DO UPDATE 
    SET item_description = EXCLUDED.item_description, item_status = EXCLUDED.item_status;"""

sql_insert_customer = """INSERT INTO customer (customer_id, customer_number, customer_name, address, 
    postal_code, city, country, country_code, telephone) VALUES (%s, LEFT(%s,50), LEFT(%s,50), LEFT(%s,100), LEFT(%s,10), LEFT(%s,50), LEFT(%s,50), LEFT(%s,10), LEFT(%s,50)) 
    ON CONFLICT (customer_id) DO UPDATE 
    SET customer_number = EXCLUDED.customer_number, customer_name = EXCLUDED.customer_name, address = EXCLUDED.address,
    postal_code = EXCLUDED.postal_code, city = EXCLUDED.city, country = EXCLUDED.country,
    country_code = EXCLUDED.country_code, telephone = EXCLUDED.telephone;"""

sql_insert_order_headers = """INSERT INTO order_header (order_id, order_number, order_date, customer_id, order_status, currency) VALUES (%s, LEFT(%s,20), %s, %s, LEFT(%s,10), LEFT(%s,10)) 
    ON CONFLICT (order_id) DO UPDATE 
    SET order_number = EXCLUDED.order_number, order_date = EXCLUDED.order_date, customer_id = EXCLUDED.customer_id,
    order_status = EXCLUDED.order_status, currency = EXCLUDED.currency;"""

sql_insert_order_lines = """INSERT INTO order_line (orderline_id, order_id, item_id, ship_date, promise_date, ordered_quantity) VALUES (%s, %s, %s, %s, %s, %s) 
    ON CONFLICT (orderline_id) DO UPDATE 
    SET order_id = EXCLUDED.order_id, item_id = EXCLUDED.item_id, ship_date = EXCLUDED.ship_date,
    promise_date = EXCLUDED.promise_date, ordered_quantity = EXCLUDED.ordered_quantity;"""

sql_insert_audit = f"""INSERT INTO audit (start_time, end_time, numberrow_treatment, status) 
VALUES (%s, %s, %s, %s);"""

# Function
def data_table(table):
    if table == 'customer':
        return df_customer.dropna(subset=['customer_number', 'customer_name', 'address', 
    'postal_code', 'city', 'country']).values.tolist()
    else:
        return eval(f"df_{table}.values.tolist()")
    
def sql_insert_table(table):
    return eval(f"sql_insert_{table}")


# Sample data for bulk insertion
data = data_table(table)
sql_insert = sql_insert_table(table)



# Function
def insert_date(sql_insert, data):

    # Replace these values with your PostgreSQL connection details 
    dbname = 'postgres'
    user = 'postgres'
    password = '31415926'
    host = 'localhost'
    port = 5432
    status = 'False'
    nunberrow_treatment = 0

    # Create a connection to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()

    start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        # Use executemany to insert the data in bulk
        cursor.executemany(sql_insert, data)

        # Commit the changes to the database
        conn.commit()

        print("Bulk insertion successful!")
        status = 'Success'
        nunberrow_treatment = len(data)
    except Exception as e:
        # If an error occurs, rollback the changes
        conn.rollback()
        print(f"Error: {e}")

    finally:
        end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Audit Log ingestion
        cursor.executemany(sql_insert_audit, [[start_time, end_time, nunberrow_treatment, status]])
        conn.commit()

        # Close the cursor and connection
        cursor.close()
        conn.close()
    

insert_date(sql_insert, data)

Bulk insertion successful!


In [241]:
# Create VIEW: Star Schema
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

cur.execute("""
    CREATE VIEW SalesHistory AS
    SELECT 
        order_line.orderline_id,
        order_header.order_id,
        item.item_id,
        customer.customer_id,
        order_line.ship_date,
        order_line.promise_date,
        order_line.ordered_quantity
    FROM order_line
    JOIN item ON item.item_id = order_line.item_id
    JOIN order_header ON order_header.order_id = order_line.order_id
    JOIN customer ON customer.customer_id = order_header.customer_id
""")

conn.commit()

cur.close()
conn.close()

DuplicateTable: relation "saleshistory" already exists


In [211]:
datetime.datetime.now()

datetime.datetime(2023, 12, 11, 2, 15, 49, 322182)

In [110]:
for order in df_order_lines.values:
    orderline_id = order[0]
    order_id = order[1]
    item_id = order[2]
    ship_date = order[3]
    promise_date = order[4]
    ordered_quantity = order[5]

In [111]:
ordered_quantity

964

In [51]:
country

'Philippines'

In [ ]:
# Backup
import psycopg2

# Replace these values with your PostgreSQL connection details 
dbname = 'postgres'
user = 'postgres'
password = '31415926'
host = 'localhost'
port = 5432

# Sample data for bulk insertion
data = [
    (value1_row1, value2_row1, value3_row1),
    (value1_row2, value2_row2, value3_row2),
    # ... add more rows as needed
]

# Construct the SQL INSERT statement
sql_insert = "INSERT INTO your_table (column1, column2, column3) VALUES (%s, %s, %s)"

def insert_date(sql_insert, data_):


# Create a connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

try:
    # Use executemany to insert the data in bulk
    cursor.executemany(sql_insert, data)

    # Commit the changes to the database
    conn.commit()

    print("Bulk insertion successful!")

except Exception as e:
    # If an error occurs, rollback the changes
    conn.rollback()
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [ ]:
# Backup Ingest: Item
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="31415926", port=5432)

cur = conn.cursor()

for item in df_item.values:
    item_id = item[0]
    item_description = item[1][:50]
    item_status = item[2][:1]

    insert_query = f"""INSERT INTO public.item (item_id, item_description, item_status) VALUES (%s, %s, %s);"""
    cur.execute(insert_query, (item_id, item_description, item_status))


conn.commit()

cur.close()
conn.close()

In [98]:
import pyarrow.csv as pv
import pyarrow.parquet as pq

filename = 'day1_files/order_headers.csv'
table = pv.read_csv(filename)
table
# pq.write_table(table, filename.replace('csv', 'parquet')

pyarrow.Table
order_id: int64
order_number: string
order_date: string
customer_id: int64
order_status: string
currency: string
----
order_id: [[1,2,3,4,5,...,20,21,22,23,24]]
order_number: [["228-75-0009","200-40-7481","632-01-9775","618-10-6504","607-11-4769",...,"441-40-7679","107-05-2829","342-13-5769","208-81-0749","744-54-9702"]]
order_date: [["2/27/2019","3/7/2019","2/13/2019","2/17/2019","4/4/2019",...,"1/25/2019","3/5/2019","3/27/2019","1/22/2019","3/4/2019"]]
customer_id: [[25,14,4,15,6,...,17,14,5,5,1]]
order_status: [["CLOSED","CLOSED","OPEN","CLOSED","CANCELLED",...,"CANCELLED","OPEN","OPEN","OPEN","CLOSED"]]
currency: [["EUR","PHP","CNY","CNY","CNY",...,"CVE","IRR","RUB","DKK","IDR"]]

In [103]:
# WORK Parquet
import pandas as pd

# Replace these with your actual file paths
csv_file_path = 'day1_files/order_headers.csv'
parquet_file_path = 'output.parquet'

# Read CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Write the DataFrame to a Parquet file
df.to_parquet(parquet_file_path, index=False)